In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [20]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+1e-3*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 23.605053
Training accuracy: 9.0%
Validation accuracy: 11.3%
Loss at step 100: 4.810731
Training accuracy: 72.3%
Validation accuracy: 70.9%
Loss at step 200: 3.962538
Training accuracy: 75.6%
Validation accuracy: 73.1%
Loss at step 300: 3.384011
Training accuracy: 77.2%
Validation accuracy: 73.9%
Loss at step 400: 2.942349
Training accuracy: 78.0%
Validation accuracy: 74.6%
Loss at step 500: 2.590778
Training accuracy: 79.1%
Validation accuracy: 74.8%
Loss at step 600: 2.302919
Training accuracy: 80.1%
Validation accuracy: 75.2%
Loss at step 700: 2.062462
Training accuracy: 81.2%
Validation accuracy: 75.7%
Loss at step 800: 1.859141
Training accuracy: 81.7%
Validation accuracy: 76.2%
Test accuracy: 84.1%


In [22]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+1e-3*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.201023
Minibatch accuracy: 5.5%
Validation accuracy: 11.5%
Minibatch loss at step 500: 2.373671
Minibatch accuracy: 82.8%
Validation accuracy: 76.4%
Minibatch loss at step 1000: 1.740792
Minibatch accuracy: 82.0%
Validation accuracy: 78.8%
Minibatch loss at step 1500: 0.936779
Minibatch accuracy: 85.2%
Validation accuracy: 79.6%
Minibatch loss at step 2000: 0.795635
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 0.877348
Minibatch accuracy: 78.9%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 0.774003
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 88.8%


In [34]:
batch_size = 128
hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta_regul = tf.placeholder(tf.float32)

  
  # Variables.
  weights1=tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_output=tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+biases1)
  logits = tf.matmul(layer1_output, weights2) + biases2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+1e-3*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer1_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(layer1_validation, weights2) + biases2)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weights2) + biases2)

In [35]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 723.226990
Minibatch accuracy: 7.8%
Validation accuracy: 34.1%
Minibatch loss at step 500: 200.248138
Minibatch accuracy: 82.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 116.552841
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 68.630722
Minibatch accuracy: 90.6%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 41.469940
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 25.199038
Minibatch accuracy: 89.8%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.470863
Minibatch accuracy: 89.8%
Validation accuracy: 86.3%
Test accuracy: 93.2%


In [38]:
batch_size = 128
hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta_regul = tf.placeholder(tf.float32)

  
  # Variables.
  weights1=tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_output=tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+biases1)
  logits = tf.matmul(layer1_output, weights2) + biases2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer1_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(layer1_validation, weights2) + biases2)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weights2) + biases2)

In [41]:
num_steps = 101
num_batches=3
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset=((step%num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 283.237244
Minibatch accuracy: 8.6%
Validation accuracy: 25.0%
Minibatch loss at step 2: 1091.256836
Minibatch accuracy: 50.0%
Validation accuracy: 35.4%
Minibatch loss at step 4: 554.105896
Minibatch accuracy: 46.9%
Validation accuracy: 53.3%
Minibatch loss at step 6: 225.853241
Minibatch accuracy: 68.0%
Validation accuracy: 67.7%
Minibatch loss at step 8: 14.497194
Minibatch accuracy: 88.3%
Validation accuracy: 74.7%
Minibatch loss at step 10: 13.850233
Minibatch accuracy: 92.2%
Validation accuracy: 75.8%
Minibatch loss at step 12: 6.530388
Minibatch accuracy: 94.5%
Validation accuracy: 73.4%
Minibatch loss at step 14: 10.723607
Minibatch accuracy: 96.1%
Validation accuracy: 75.2%
Minibatch loss at step 16: 8.140869
Minibatch accuracy: 96.1%
Validation accuracy: 74.8%
Minibatch loss at step 18: 10.856525
Minibatch accuracy: 96.1%
Validation accuracy: 75.5%
Minibatch loss at step 20: 0.284326
Minibatch accuracy: 98.4%
Validation accuracy: 75.9%
Mi

In [48]:
batch_size = 128
hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta_regul = tf.placeholder(tf.float32)

  
  # Variables.
  weights1=tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_output=tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+biases1)
  drop1 = tf.nn.dropout(layer1_output, 0.5)
  logits = tf.matmul(drop1, weights2) + biases2
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer1_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(layer1_validation, weights2) + biases2)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(layer1_test, weights2) + biases2)

In [49]:
num_steps = 101
num_batches=3
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #offset=((step%num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    offset=step%num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 518.643311
Minibatch accuracy: 8.6%
Validation accuracy: 34.1%
Minibatch loss at step 2: 828.434998
Minibatch accuracy: 39.8%
Validation accuracy: 44.8%
Minibatch loss at step 4: 418.839478
Minibatch accuracy: 67.2%
Validation accuracy: 53.9%
Minibatch loss at step 6: 135.794510
Minibatch accuracy: 78.1%
Validation accuracy: 63.9%
Minibatch loss at step 8: 9.027114
Minibatch accuracy: 96.1%
Validation accuracy: 66.4%
Minibatch loss at step 10: 15.753027
Minibatch accuracy: 95.3%
Validation accuracy: 68.1%
Minibatch loss at step 12: 3.003676
Minibatch accuracy: 98.4%
Validation accuracy: 65.8%
Minibatch loss at step 14: 16.365269
Minibatch accuracy: 97.7%
Validation accuracy: 68.4%
Minibatch loss at step 16: 2.782938
Minibatch accuracy: 96.9%
Validation accuracy: 66.1%
Minibatch loss at step 18: 19.632530
Minibatch accuracy: 97.7%
Validation accuracy: 66.4%
Minibatch loss at step 20: 7.465196
Minibatch accuracy: 96.9%
Validation accuracy: 68.2%
Mini

In [58]:
batch_size = 128
hidden_nodes1=1024
hidden_nodes2=100

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)
  #beta_regul = tf.placeholder(tf.float32)

  
  # Variables.
  weights1=tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1],stddev=np.sqrt(2.0 / (image_size * image_size))))
  weights2=tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2],stddev=np.sqrt(2.0 /hidden_nodes1)))
  weights3 = tf.Variable(
    tf.truncated_normal([hidden_nodes2, num_labels],stddev=np.sqrt(2.0 /hidden_nodes2)))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_output=tf.nn.relu(tf.matmul(tf_train_dataset, weights1)+biases1)
  layer2_output=tf.nn.relu(tf.matmul(layer1_output, weights2)+biases2)
  logits = tf.matmul(layer2_output, weights3) + biases3
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+1e-3*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(weights3))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  layer1_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  layer2_validation = tf.nn.relu(tf.matmul(layer1_validation, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(layer2_validation, weights3) + biases3)
  layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  layer2_test = tf.nn.relu(tf.matmul(layer1_test, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(layer2_test, weights3) + biases3)

In [60]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.281775
Minibatch accuracy: 7.8%
Validation accuracy: 31.9%
Minibatch loss at step 500: 0.940659
Minibatch accuracy: 89.1%
Validation accuracy: 85.7%
Minibatch loss at step 1000: 0.874211
Minibatch accuracy: 83.6%
Validation accuracy: 86.6%
Minibatch loss at step 1500: 0.556904
Minibatch accuracy: 93.0%
Validation accuracy: 87.9%
Minibatch loss at step 2000: 0.503312
Minibatch accuracy: 93.8%
Validation accuracy: 88.0%
Minibatch loss at step 2500: 0.543016
Minibatch accuracy: 89.8%
Validation accuracy: 88.5%
Minibatch loss at step 3000: 0.560205
Minibatch accuracy: 88.3%
Validation accuracy: 88.6%
Minibatch loss at step 3500: 0.563279
Minibatch accuracy: 88.3%
Validation accuracy: 88.9%
Minibatch loss at step 4000: 0.448858
Minibatch accuracy: 92.2%
Validation accuracy: 89.0%
Minibatch loss at step 4500: 0.435801
Minibatch accuracy: 90.6%
Validation accuracy: 89.3%
Minibatch loss at step 5000: 0.497764
Minibatch accuracy: 90.6%
Validation accuracy